# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰을 작성하고 실행하는 과정을 설명합니다.
- 난이도: 매우 낮음
- 학습 시간: 10분

⚠︎ 주의사항: 코드를 변경하며 실행하고 싶으신 분들은, 사본을 생성한 후 작업해주십시오.

## 룰 작성하기
이 부분에서는 기존의 IFC에 대한 검사를 목적으로 개발된 IFCRule 및 RuleController를 사용하지 않고, 새로이 개발한 RuleUnit 클래스를 활용하여 룰을 작성하는 방법을 다룹니다.<br><br>
본 예시에서 사용하는 기준은 KDS 24 14 21 4.6.5.1(5)을 바탕으로 작성되었습니다.

먼저, Rule 작성에 필요한 클래스 및 함수를 불러오기 위하여, Github를 통해 Tomok이라는 Repository를 Clone합니다.

In [1]:
!git clone https://github.com/KU-HIAI/tomok.git

fatal: 대상 경로가('tomok') 이미 있고 빈 디렉터리가 아닙니다.


Repository를 Clone하였다면, 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.

In [2]:
cd tomok

/Users/jaewooklee/Documents/Github/tomok/tomok/tomok


다음 코드는 KDS 24 14 21 4.6.5.1(5)의 메타정보에 해당하는 변수 정의 부분입니다.

In [3]:
priority = 1
author = 'Jaewook Lee'
ref_code = 'KDS 24 14 21 4.6.5.1 (5)'
ref_date = '2023-06-28'
title = '바닥판 최소두께'
description = """
    콘크리트교 설계기준(한계상태설계법)
    4. 설계
    4.6 부재 상세
    4.6.5 교량의 콘크리트 바닥슬래브
    4.6.5.1 일반 사항
    (5)
"""
content = """
    #### 4.6.5.1 일반 사항
(5) 특별히 요구되지 않는 한, 콘크리트 바닥판은 홈 또는 마모 방지 층의 두께를 뺀 판 최소 두께는 220mm 보다 작아서는 안 된다. 프리스트레스트 콘크리트 바닥판의 최소두께는 200mm 이상이어야 한다. 바닥판의 최소 피복 두께는 4.4의 규정을 따라야 한다.
"""
flowchart = """
    flowchart TD
    %% Nodes
    A["ProfileSectionSlabThickness(SLATHI)"]
    B["ReviewValueSlabMinThickness(SLAMINTHI)"]
    C{"SLATHI > SLAMINTHI"}
    D[True]
    E[False]

    %% Edges
    A --> C
    B --> C
    C --True--> D
    C --False--> E
"""

#### 함수 정의
다음으로 RuleUnit에 작성될 룰의 실행 내용을 다루는 함수에 대해 알아보겠습니다.<br><br>
개편된 RuleUnit 클래스는 룰의 메타 정보와 실제 실행 부분만을 포함하기 때문에, 실제 실행에 해당하는 함수만을 가지고 있습니다.<br><br>
다음 코드는 KDS 24 14 21 4.6.5.1(5) 내에 정의된 실행 함수인 slab_min_thickness 입니다.<br><br><br><br>
실행 함수의 이름은 작업자가 작성하는 룰에 맞게 임의로 설정하여 작성하게 됩니다.

In [4]:
@tomok.rule_method
def slab_min_thickness(SLATHI, SLAMINTHI) -> bool:
    """홈 또는 마모 방지 층의 두께를 뺀 콘크리트 바닥판의 두께가 최소 두께를 만족하는 지의 여부
        
    Args:
        SLATHI (int): 교량의 콘크리트 바닥판의 홈 또는 마모 방지 층의 두께를 뺀 판의 두께
        SLAMINTHI (int): 교량의 콘크리트 바닥판의 홈 또는 마모 방지 층의 두께를 뺀 판의 최소 두께

    Returns:
        bool: 콘크리트교 설계기준(한계상태설계법) 4.6.5.1 일반 사항의 항목 (5)의 통과 여부
    """
        
    if SLATHI > SLAMINTHI:
        return True
    else:
        return False

NameError: name 'tomok' is not defined

RuleUnit의 실행 함수는 기존의 OKNGResult 클래스를 활용하지 않고, 해당 실행함수의 기준을 만족하였는지의 여부를 bool 타입의 값으로 반환합니다.

## 룰 작성해보기
지금까지의 내용을 바탕으로 RuleUnit 클래스를 상속하는 룰을 작성해봅시다.


In [8]:
from tomok.core.rule_ifc import RuleIFC
from tomok.ifc.reader import IFCReader
from tomok.ifc.entity import Product
from notebooks.KDS_24_14_21_4_6_5_1_5 import KDS_24_14_21_4_6_5_1_5
from tomok.core.util import *
from typing import List

# 23.10.24 - 기존에는 IFCReader가 부재 종류를 기반으로 타겟 부재를 검색하였으나, guid를 기반으로 타겟 부재를 검색하도록 변경되었습니다.
#            이에 따라 RuleIFC 클래스 호출 및 부재 검색에 사용되는 인자가 바뀌었으니 참고 부탁드립니다.

# 실제 RuleIFC 작성 시 클래스 명을 임의로 수정하시면 됩니다.
class RuleIFC_KDS_24_14_21_4_6_2_1_5(RuleIFC):
    # RuleIFC에서 검토할 RuleUnit을 이 부분에서 불러옵니다.
    ruleunit_1 = KDS_24_14_21_4_6_5_1_5()

    def __init__(self,
                 reader: IFCReader):
        self.reader = reader

    # 실제 RuleIFC 작성 시 수정해야 할 부분입니다.
    # IFC 파일에서 guid를 기반으로 부재를 검색하는 부분입니다.
    def retrieve_entities(
        self,
        guid: str):
        logging_normal(content=f"Target GUID: {guid}", tag="SEARCH")
        self.reader.get_products()
        try:
            target_entitiy = [self.reader.get_product_by_guid(guid)]
            logging_info(content="retrieved entity = {}".format(target_entitiy), tag="FIND")
        except:
            logging_info(content="Target Entity NOT FOUND!!!", tag="ALART")
            return []
        return target_entitiy

    # 실제 RuleIFC 작성 시 수정해야 할 부분입니다.
    # RuleIFC에서 불러온 부재들에 적용할 기준을 작성하는 부분입니다.
    # 작성될 내용은 RuleIFC로 구현하는 건설기준에 따라 상이합니다.
    def process(cls,
                entity: Product,
                ):
        # 본 예제에서는, 바닥판의 최소 두께에 대한 기준을 검토하므로 ifc 파일에서 해당 값을 가져와 RuleUnit의 실행 함수에 전달합니다.
        ruleunit_name = cls.ruleunit_1.ref_code
        logging_normal("{}".format(ruleunit_name))
        try:
            psets = entity.get_psets() # entity의 pset 가져오기
            for pset in psets:
                if pset.has_property("ReviewValueSlabMinThickness"): # 최소 두께 값 찾아서
                    SLAMINTHI = pset.ReviewValueSlabMinThickness
                    print(SLAMINTHI)

            psets = entity.IfcProfileDef.get_psets()
            for pset in psets:
                if pset.has_property("ProfileSectionSlabThickness"):
                    SLATHI = pset.ProfileSectionSlabThickness
                    print(SLATHI)

            result = cls.ruleunit_1.slab_min_thickness(SLATHI, SLAMINTHI)
            if result:
                logging_clear(content="SLATHI={} / SLAMINTHI={} ... PASSED".format(SLATHI, SLAMINTHI), tag="RESULT")
            # 실행 결과는 통과했는지에 대한 여부를 True or False로 반환합니다.
            return result
        except:
            logging_warn(content="FAILED", tag="RESULT")
            return False

RuleUnit 클래스를 상속하여 클래스로 작성한 룰은 아래의 코드와 같이 인스턴스로 생성할 수 있습니다.

In [9]:
temp_ruleifc = RuleIFC_KDS_24_14_21_4_6_2_1_5(
    reader=IFCReader("/Users/jaewooklee/Documents/Github/tomok/tomok/대성요청확장파일_v4 (1).ifc")
)

아래와 같이 임의로 슬래브의 두께와, 최소로 충족하여야 하는 두께가 주어질 때, 룰의 실행 함수가 올바르게 작동하는지 확인할 수 있습니다.

In [ ]:
entities = temp_ruleifc.retrieve_entities(
    guid='3$Y1qZFGzBO9A70r9J2$Md'
)

In [10]:
temp_ruleifc.reader.get_products()

{'entity_instances': [[56/IfcSite/] [pset: Site] [mat: Site] [prof: Site] [rule:None]], 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': [], 'IfcSite': {'entity_instances': [[25188/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None], [57/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None]], 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': []}}
{'entity_instances': [[111/IfcWall/] [pset: Beam-43431] [mat: Beam-43431] [prof: Beam-43431] [rule:None]], 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': [], 'IfcWall': {'entity_instances': [[105/IfcRelDefinesByProperties/] [pset: Basic] [mat: None] [prof: None] [rule:None], [98/IfcRelAssociatesMaterial/] [pset: None] [mat: None] [prof: None] [rule:None], [25183/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None], [109/IfcRelDefinesByPropert

[#56=IfcSite('0EwlUNhML1whKrVitXM4AV',$,'Site',$,$,#9,$,$,$,(0,0,0,0),(0,0,0,0),0.000000000000000,$,$),
 #111=IfcWall('2n3$47Oxz8_uN$d_hkcPwN',$,'Beam-43431',$,$,#9,#110,$,.SOLIDWALL.),
 #242=IfcBeam('0ZCqvdQhLE28JaY93BNupT',$,'Beam-42895',$,'BASE_PLATE',#9,#241,$,.BEAM.),
 #270=IfcBeam('3euxCoEF99OBR8595dqh2P',$,'Beam-43428',$,$,#9,#269,$,.BEAM.),
 #843=IfcBeam('3Ywf2eyYnAYgJjIEs6obpt',$,'Beam-43429',$,$,#9,#842,$,.BEAM.),
 #1605=IfcReinforcingBar('0W6TUt0RL92e0XMLDkp6Uq',$,'3-1',$,$,#9,#1604,$,'SD400',15.900000000000000,198.556509688508896,$,.USERDEFINED.,$),
 #2367=IfcReinforcingBar('2SdbuOtqf3D8lb_pU0WEg5',$,'3-3',$,$,#9,#2366,$,'SD400',15.900000000000000,198.556509688508896,$,.USERDEFINED.,$),
 #3129=IfcReinforcingBar('2JeCBZQ2j9ePuBaf7wiKTG',$,'3-10',$,$,#9,#3128,$,'SD400',15.900000000000000,198.556509688508896,$,.USERDEFINED.,$),
 #3891=IfcReinforcingBar('12UwuV2kf65wYPmQDKpOwg',$,'3-9',$,$,#9,#3890,$,'SD400',15.900000000000000,198.556509688508896,$,.USERDEFINED.,$),
 #5355=IfcR

In [ ]:
Rule_Review_Result = tempRule.slab_min_thickness(SlabThickness, SlabMinThickness)

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

NameError: name 'tempRule' is not defined

아래의 코드를 통해, 룰의 content의 markdown 렌더링 결과를 확인할 수 있습니다.

In [1]:
from ruleifc_kds142054_040305_01 import RuleIFC_KDS124054_040305_01
from tomok.ifc import IFCReader

In [13]:
IFCReader(ifc_filepath="/Users/jaewooklee/Documents/Github/tomok/tomok/Extradosed Bridge_extended.ifc").get_products()

{'entity_instances': [[95/IfcBuilding/] [pset: ] [mat: ] [prof: ] [rule:None]], 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': []}


KeyError: 'IfcBuilding'

In [15]:
reader = IFCReader("../tomok/notebooks/psc_girder_bridge_final.ifc")
reader.get_products()

{'entity_instances': [[142/IfcBuilding/] [pset: ] [mat: ] [prof: ] [rule:None]], 'pset': {}, 'material': {}, 'profile': {}, 'id2rel_class': {}, 'rule_decision': [], 'rule_indirect': [], 'IfcBuilding': {'entity_instances': [[1094/IfcRelDefinesByProperties/] [pset: Pset_BuildingSystemCommon] [mat: None] [prof: None] [rule:None], [1084/IfcRelDefinesByProperties/] [pset: Pset_BuildingCommon] [mat: None] [prof: None] [rule:None], [1068/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None], [1091/IfcRelDefinesByProperties/] [pset: Pset_BuildingStoreyCommon] [mat: None] [prof: None] [rule:None], [1064/IfcRelAggregates/] [pset: None] [mat: None] [prof: None] [rule:None], [1088/IfcRelDefinesByProperties/] [pset: Pset_BuildingElementProxyCommon] [mat: None] [prof: None] [rule:None]], 'pset': {'Pset_BuildingSystemCommon': #1082=IfcPropertySet('2_wasMqT55PPxcC$UCt6vB',#42,'Pset_BuildingSystemCommon',$,(#175)), 'Pset_BuildingCommon': #1074=IfcPropertySet('27PCKGLxT4mxtVBOQ6mgBW',#42,

[#142=IfcBuilding('3jFItWokXEReTKxuco5LtJ',#42,'',$,$,#33,$,'',.ELEMENT.,$,$,#138),
 #155=IfcBuildingStorey('3jFItWokXEReTKxubDwgCL',#42,'1F',$,'레벨:삼각형 헤드',#153,$,'1F',.ELEMENT.,0.),
 #171=IfcSite('3jFItWokXEReTKxuco5LtG',#42,'Default',$,$,#170,$,$,.ELEMENT.,(37,33,59,529418),(126,58,40,678710),0.,$,$),
 #335=IfcBeam('3tke7nC1HEAgqfTM8BlJS6',#42,'2C:2C 고원희 선배님:376319',$,'2C:2C 고원희 선배님',#333,#324,'376319',.BEAM.),
 #413=IfcBeam('3tke7nC1HEAgqfTM8BlJLl',#42,'2C:2C 고원희 선배님:376726',$,'2C:2C 고원희 선배님',#412,#405,'376726',.BEAM.),
 #463=IfcBeam('3tke7nC1HEAgqfTM8BlJLI',#42,'2C:2C 고원희 선배님:376747',$,'2C:2C 고원희 선배님',#462,#455,'376747',.BEAM.),
 #513=IfcBeam('3tke7nC1HEAgqfTM8BlJKo',#42,'2C:2C 고원희 선배님:376779',$,'2C:2C 고원희 선배님',#512,#505,'376779',.BEAM.),
 #563=IfcBeam('3tke7nC1HEAgqfTM8BlJKG',#42,'2C:2C 고원희 선배님:376809',$,'2C:2C 고원희 선배님',#562,#555,'376809',.BEAM.),
 #647=IfcBeam('3tke7nC1HEAgqfTM8BlKci',#42,'콘크리트-직사각형 보:900 x 2500:377685',$,'콘크리트-직사각형 보:900 x 2500',#646,#639,'377685',.BEAM.),
 #720

In [ ]:
ruleifc = RuleIFC_KDS124054_040305_01(
    reader=IFCReader(ifc_filepath="./대성요청확장파일_v4.ifc")
    )

NameError: name 'RuleIFC_KDS124054_040305_01' is not defined

In [ ]:
entities = ruleifc.retrieve_entities(guid="2v6Vtw6InEjOXL1u9Ht7jd")

In [ ]:
ruleifc.process(entity=entities)

[ 23-11-14 13:10:39 ] : RuleIFC: process START...
단일 부착식 앵커: Fail
부착식 앵커 그룹: Fail
